<a href="https://colab.research.google.com/github/ash1998win/uttarakhhand/blob/main/uttarakhand_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT THE LIBRARIES

In [36]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import pandas as pd
np.random.seed(400)

In [37]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
df = pd.read_csv('Uttarakhand.csv')

In [39]:
df.head()

,tweet
0,Horrible news out of #Uttarakhand
1,Prayer for #Uttarakhand 🙏🙏
2,Ohhh God Please Save #Uttarakhand
3,Extremely Sad To Hear About The #Uttarakhand G...
4,Prayers for #Uttarakhand 🙏🙏


In [40]:
stemmer=SnowballStemmer('english')

Preprocess

In [41]:
def lemmatize_stem(text):
  return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='v'))
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_punctuation
my_filter = [
  lambda x: x.lower(), strip_tags, strip_punctuation,
  strip_multiple_whitespaces, strip_punctuation,
  remove_stopwords, strip_short, stem_text]

def preprocess(text):
  
  result=[]

  for token in preprocess_string(text,filters=my_filter):
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token)>=3 :
      result.append(lemmatize_stem(token))
  return result

In [42]:
df['index']=df.index


In [43]:
df.head()

,tweet,index
0,Horrible news out of #Uttarakhand,0
1,Prayer for #Uttarakhand 🙏🙏,1
2,Ohhh God Please Save #Uttarakhand,2
3,Extremely Sad To Hear About The #Uttarakhand G...,3
4,Prayers for #Uttarakhand 🙏🙏,4


In [44]:
df_num=1234
df_sample=df[df['index']==df_num].values[0][0]
print('orgdoc:')
words=[]
for word in df_sample.split(' '):
  words.append(word)
print(words)
print('\n\nTokenized and lemmatized data:')
print(preprocess(df_sample))

orgdoc:
['#Uttarakhand', 'Glacier', 'Disaster\n\nWATCH', '|', 'ITBP', 'teams', 'in', 'rescue', 'mission', 'near', 'Tapovan\n\nLIVE', 'updates', '👇…', 'https://t.co/NpiomHVFMa']


Tokenized and lemmatized data:
['uttarakhand', 'glacier', 'disast', 'watch', 'itbp', 'team', 'rescu', 'mission', 'near', 'tapovan', 'live', 'updat', 'http', 'npiomhvfma']


In [45]:
processed_df=df['tweet'].map(preprocess)

In [46]:
processed_df[:10]

0                          [horribl, new, uttarakhand]
1                                [prayer, uttarakhand]
2                       [ohhh, god, save, uttarakhand]
3    [extrem, sad, hear, uttarakhand, glacier, brea...
4                                [prayer, uttarakhand]
5                                [prayer, uttarakhand]
6    [alert, uttarakhand, emerg, number, 1070, 9557...
7    [heavi, devast, glacier, breakag, chamoli, utt...
8    [wish, stai, safe, uttarakhand, tragedi, uttar...
9                     [lord, shiva, save, uttarakhand]
Name: tweet, dtype: object

In [47]:
dict_df=gensim.corpora.Dictionary(processed_df)

In [48]:
bow_corpus=[dict_df.doc2bow(doc) for doc in processed_df]

In [49]:
bow_corpus[1234]

[(2, 1),
 (10, 1),
 (13, 1),
 (57, 1),
 (95, 1),
 (97, 1),
 (117, 1),
 (220, 1),
 (225, 1),
 (256, 1),
 (582, 1),
 (601, 1),
 (886, 1),
 (2199, 1)]

In [50]:
from gensim import corpora, models
tfidf=models.TfidfModel(bow_corpus)
print(tfidf)

TfidfModel(num_docs=8146, num_nnz=72649)


In [51]:
corpustfidf=tfidf[bow_corpus]
print(corpustfidf[2])

[(2, 0.021177955889248794), (4, 0.34995096742016835), (5, 0.833955461479987), (6, 0.42615032893773935)]


In [52]:
lda_model=gensim.models.LdaMulticore(bow_corpus,num_topics=30,id2word=dict_df,passes=2,workers=2)

In [53]:
for idx , topic in lda_model.print_topics(-1):
  print(f"Topic: {idx}\nWords: {topic}")
  print('\n')

Topic: 0
Words: 0.077*"uttarakhand" + 0.055*"http" + 0.035*"prayer" + 0.013*"glacier" + 0.012*"🙏🏻🙏🏻" + 0.012*"chamoli" + 0.011*"prai" + 0.008*"break" + 0.007*"tsrawatbjp" + 0.007*"real"


Topic: 1
Words: 0.042*"http" + 0.035*"uttarakhand" + 0.035*"chamoli" + 0.027*"flood" + 0.027*"ganga" + 0.025*"dhauli" + 0.022*"joshimath" + 0.021*"district" + 0.016*"river" + 0.016*"glacier"


Topic: 2
Words: 0.069*"http" + 0.063*"uttarakhand" + 0.028*"area" + 0.025*"help" + 0.022*"need" + 0.022*"affect" + 0.021*"stick" + 0.020*"kind" + 0.016*"contact" + 0.015*"chamoli"


Topic: 3
Words: 0.108*"uttarakhand" + 0.043*"http" + 0.036*"prai" + 0.019*"prayer" + 0.017*"peopl" + 0.015*"chamoli" + 0.012*"flood" + 0.012*"amp" + 0.007*"safeti" + 0.007*"glacier"


Topic: 4
Words: 0.137*"uttarakhand" + 0.055*"prayer" + 0.049*"http" + 0.045*"chamoli" + 0.027*"prai" + 0.027*"1070" + 0.026*"9557444486" + 0.021*"emerg" + 0.020*"har" + 0.014*"helplin"


Topic: 5
Words: 0.114*"uttarakhand" + 0.053*"http" + 0.041*"glacie

In [54]:
lda_model_tfidf=gensim.models.LdaMulticore(corpustfidf,num_topics=30,id2word=dict_df,passes=2,workers=2)

In [55]:
for idx , topic in lda_model_tfidf.print_topics(-1):
  print(f"Topic: {idx}\nWords: {topic}")
  print('\n')

Topic: 0
Words: 0.012*"prayer" + 0.008*"chamoli" + 0.007*"prai" + 0.007*"new" + 0.006*"flood" + 0.006*"safeti" + 0.006*"glacier" + 0.005*"peopl" + 0.005*"burst" + 0.005*"http"


Topic: 1
Words: 0.009*"chamoli" + 0.008*"prayer" + 0.008*"prai" + 0.007*"safe" + 0.006*"flood" + 0.005*"help" + 0.005*"1070" + 0.005*"tsrawatbjp" + 0.005*"god" + 0.005*"http"


Topic: 2
Words: 0.012*"prayer" + 0.006*"chamoli" + 0.005*"http" + 0.005*"joshimath" + 0.005*"glacier" + 0.005*"peopl" + 0.005*"flood" + 0.005*"itbp" + 0.004*"uttarakhand" + 0.004*"water"


Topic: 3
Words: 0.010*"prai" + 0.010*"water" + 0.009*"level" + 0.007*"chamoli" + 0.005*"prayer" + 0.005*"glacier" + 0.005*"rise" + 0.004*"http" + 0.004*"flood" + 0.004*"amp"


Topic: 4
Words: 0.006*"help" + 0.006*"chamoli" + 0.005*"center" + 0.005*"http" + 0.005*"disast" + 0.005*"contact" + 0.005*"oper" + 0.005*"area" + 0.004*"flood" + 0.004*"har"


Topic: 5
Words: 0.018*"🙏🏼🙏🏼" + 0.009*"prayer" + 0.007*"chamoli" + 0.007*"sad" + 0.006*"glacier" + 0.006*

what is the helpline number?
9557444486,1070
what happened in uttarakhand?
In uttrakhand district of chamoli a glacial burst occured which led to flooding of dam and several people were affected
How many people are affected?
 150
 which rivers water level rose high?
 Ganga



